In [70]:
using Iterators
using Pipe

macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

In [71]:
using RecursiveAutoencoders
#using WordEmbeddings
using UnfoldingRAE

import UnfoldingRAE.NoSide
import UnfoldingRAE.Left
import UnfoldingRAE.Right


In [72]:
using WordEmbeddings
LL,word_indexes, indexed_words = load_word2vec_embeddings("../../Resources/example_code/word2vec/GoogleNews-vectors-negative300.bin", 15000)
size(LL) |> println
word_indexes |> typeof |> println
indexed_words |> typeof |> println

(300,15000)
Dict{String,Int64}
Array{String,1}


In [73]:
training_trees = open("training_sents.jsz","r") do fs
    deserialize(fs)
end;
@assert length(@pipe training_trees |> filter(x->typeof(x)===UTF8String,_))==0
@pz training_trees

training_trees		Array{Any,1}	(3913,)


In [118]:
type M_UnfoldLeaf
    leaf::UnfoldLeaf
    score::Number
    index::Int
end

In [124]:
using Memoize
function evaluate_morph_unfoldings(rae::RAE, act::FoldData, orders::Vector{FoldData})
    @memoize Dict function recon(ĉ::Embedding)
        sims= WordEmbeddings.neighbour_sims(ĉ, rae.L)
        score, index = findmax(sims)
        c = rae.L[:,index] 
        score, c, index
    end
    
    function morph_unfold{T}(_, ĉ::Embedding, parent, ::Type{T}, depth)
        score,c, index = recon(ĉ)
        score, M_UnfoldLeaf(UnfoldLeaf{T}(ĉ, parent, c, depth), score,index)
    end

    function morph_unfold{T}(order::FoldData, p_in::Embedding, parent, ::Type{T}, depth::Int)
        ĉ_i, ĉ_j = reconstruct(rae,p_in)
        data = UnfoldData{T}(p_in, parent, ĉ_i, ĉ_j,depth)

        left_score, left = morph_unfold(order.left, ĉ_i, data, Left, depth+1)
        right_score, right= morph_unfold(order.right, ĉ_j, data, Right, depth+1)
        [left_score, right_score], [left; right]
    end
    scores = Array(Number,length(orders))
    unfold_leafs = Array(Array{M_UnfoldLeaf}, length(orders))
    for ii in 1:length(orders)
        order = orders[ii]
        leaf_scores, unfold_leafs[ii] = morph_unfold(order,act.p_out, act, NoSide,0)
        scores[ii]=mean(leaf_scores)
    end
    
    
    scores,unfold_leafs
    
    
end

evaluate_morph_unfoldings (generic function with 1 method)

In [125]:
function make_blank_FoldData(tree::(Any,Any))
    function make_blank_FoldData(leaf::Any)
        [NaN]
    end
    
    left = make_blank_FoldData(tree[1])
    right = make_blank_FoldData(tree[2])
    fd = FoldData([NaN], left, right)
    fd
end

make_blank_FoldData (generic function with 2 methods)

In [126]:
o1=make_blank_FoldData((1,(1,2)))
o2=make_blank_FoldData(((1,2),1))

FoldData([NaN],FoldData([NaN],[NaN],[NaN]),[NaN])

In [127]:
tree = training_trees[1]
rae_os = RAE(LL,word_indexes,indexed_words,0.1);
fold_tree = fold(rae_os, tree)
scores,unfold_leafs = evaluate_morph_unfoldings(rae_os, fold_tree, [o1,o2]);


In [129]:
score, index = findmax(scores)
recon = map(mleaf->rae_os.indexed_words[mleaf.index], unfold_leafs[index])

3-element Array{ASCIIString,1}:
 "Wave"    
 "Patients"
 "Colombia"

In [108]:
scores

2-element Array{Number,1}:
 0.225472
 0.20878 